<a href="https://colab.research.google.com/github/soogle25/yolov5/blob/master/method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#YOLOv5の学習手順（dir=./yolov5/yolo）

```
python train.py --img 416 --batch 16 --epochs 200 --data dataset0_416.yaml --cfg yolov5s.yaml --weights yolov5s.pt --patience 1000 --name smd_v5s_dataset0_416_b16 --conf-thres 0.5 --iou-thres 0.5
```

###引数説明

--img：学習画像サイズ

--batch：バッチサイズ

--epochs：学習回数

--data：train画像とvalidation画像のディレクトリの指定とクラス数およびクラス名を指定したyamlファイル

（例：smd_v5s(x)_dataset0(1,2,3,4,5)_teian4(teian7)_C3ECA(C3SE,C3CBAM)_416(512,640)_b16(8,4)）

--cfg：学習用モデル（アーキテクチャ）のyamlファイル(./models/)

(例：yolov5s_teian4_C3ECA)←data指定と同様

--weights：転移学習するための事前学習済み重み

--patience：早期学習打ち切り防止のためのepoch指定で利用

--name：学習後重みの名前指定

--conf-thres：信頼度しきい値

--iou-thres：IoU値しきい値

※論文の提案法1がteian4、提案法2がteian7、提案法3がC3ECAなどのC3+CA名、提案法4がC3ECA2などのC3+CA名+2に該当

#YOLOv5の検出手順（dir=./yolov5/yolo）(通常)

```
python detect.py --weights runs/train/smd_v5s_dataset0_416_b16/weights/best.pt --imgsz 416
```

###引数説明

--weights：検出用学習済重み

--imgsz：画像サイズ

#mAPとwidthごとのRecallの測定（dir=./yolov5/yolo）

##まず検出時のtxtファイルを作成し、mAPフォルダへ保存

###(OnlyBoat/smd_dataset0)にdetectが存在する場合は削除する必要がある（mAP&Recall/smd_dataset0）

```
shutil.rmtree('./detect')
```

```
!python detect.py --weights runs/train/smd_v5s_dataset0_416_b16/weights/best.pt --source ../../mAP&Recall/smd_dataset0/images --project ../../mAP&Recall/smd_dataset0/detect --imgsz 416 --save-txt --save-conf 
```

###引数説明

--weights：検出に用いる重み

--source：検証用データセットの指定

--project：検出結果の保存先の指定

--imgsz：検出画像サイズ

--save-txt：検出結果をテキストファイルで保存

--save-conf:保存したテキストファイルに信頼度を保存

※学習時のデータセットのvalidation画像とmAP&Recallのtest画像はデータセット名ごとに統一してあるため、学習時と検証時のデータセット名は合わせる必要がある。学習に用いていない画像であれば新しく検証用データセットを作ることも可能

```
#dataset指定
number=0

#confとpredの位置変換(mAP計算のデータセットに合わせるため)
import re
import numpy as np
from glob import glob
```
##（dir=./mAP&Recall/smd_dataset'+str(number)+'/detect/exp/labels'))）
```
shutil.rmtree('../../../detections')
os.mkdir('../../../detections')

for file in glob('*.txt'):
  if file:
    num=np.loadtxt(file) 
    #print(file)
    #confとpredの入れ替え
    #行列の要素が2以上
    if num.ndim==2:
      change=num[:,[0,5,1,2,3,4]]
      #Classをint型に変換
      for i in range(len(num)):
        Class=int(change[i][0])
        conf=str(change[i][1])
        pred_1=str(change[i][2])
        pred_2=str(change[i][3])
        pred_3=str(change[i][4])
        pred_4=str(change[i][5])
        input=str(Class)+" "+str(conf)+" "+str(pred_1)+" "+str(pred_2)+" "+str(pred_3)+" "+str(pred_4)+"\n"
        with open("../../../detections/"+str(file), mode='a') as ff:
          ff.write(input)   

    #行列の要素が１
    else:
      change=num[[0,5,1,2,3,4]]
      chenge=np.array(change).reshape(1,6)
      Class=int(change[0])
      conf=str(change[1])
      pred_1=str(change[2])
      pred_2=str(change[3])
      pred_3=str(change[4])
      pred_4=str(change[5])
      input=str(Class)+" "+str(conf)+" "+str(pred_1)+" "+str(pred_2)+" "+str(pred_3)+" "+str(pred_4)
      with open("../../../detections/"+str(file), mode='a') as ff:
        ff.write(input)
```
#mAPとRecallの計算(dir=./mAP&Recall/smd_dataset'+str(number))
```
!python main.py  --gt_dir groundtruths --det_dir detections --imgsz 1920

```
